In [1]:
import tensorflow as tf
import os
import random
import sys
import numpy as np
from PIL import Image

In [2]:
num_train = 8000
captcha_img_dir = './captcha-images/'
tfrecord_dir = './captcha-tfrecords/'

In [3]:
def mkdir(tfrecord_dir=tfrecord_dir):
    if not os.path.exists(tfrecord_dir):
        os.mkdir(tfrecord_dir)

In [4]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [5]:
def get_img_name(captcha_img_dir=captcha_img_dir):
    img_name = []
    for imgname in os.listdir(captcha_img_dir):
        if imgname.endswith('jpg'):
            img_path = os.path.join(captcha_img_dir, imgname)
            img_name.append(img_path)
    return img_name

In [6]:
def image_to_tfexample(img_data, label0, label1, label2, label3):
    return tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(img_data),
        'label0': _int64_feature(label0),
        'label1': _int64_feature(label1),
        'label2': _int64_feature(label2),
        'label3': _int64_feature(label3)
    }))

In [8]:
def convert_data(split_name, filenames, tfrecord_dir=tfrecord_dir):
    assert split_name in ['train', 'test']

    with tf.Session() as sess:
        # tfrecord file path
        output_filename = os.path.join(tfrecord_dir, split_name + '.tfrecords')
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            for i, filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image {}/{}'.format(i + 1, len(filenames)))
                    sys.stdout.flush()

                    # read image
                    image_data = Image.open(filename)

                    # resize image to 224x224 (size of inputs in AlexNet)
                    image_data = image_data.resize((224, 224))
                    
                    # gray scale
                    image_data = np.array(image_data.convert('L'))

                    # convert to bytes formayt
                    image_data = image_data.tobytes()

                    # get labels
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        if labels[j].isdigit():
                            num_labels.append(int(labels[j]))
                        elif labels[j].islower():
                            num_labels.append(int(ord(labels[j]) - (ord('a') - 10)))
                        elif labels[j].upper():
                            num_labels.append(int(ord(labels[j]) - (ord('A') - 36)))

                    example = image_to_tfexample(image_data, num_labels[0], num_labels[1], num_labels[2], num_labels[3])
                    tfrecord_writer.write(example.SerializeToString())
                    
                except IOError as e:
                    print('Could not read: {}'.format(filename))
                    print('Error: {}'.format(e))
                    print('Skip it!\n')
                    
    sys.stdout.write('\n')
    sys.stdout.flush()

In [9]:
if __name__ == "__main__":
    mkdir()
    photo_filenames =get_img_name(captcha_img_dir)
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[:num_train]
    testint_filenames = photo_filenames[num_train:]
    convert_data('train', training_filenames, tfrecord_dir)
    convert_data('test', testint_filenames, tfrecord_dir)
    print("TFRecord file created！")

>> Converting image 8000/8000
>> Converting image 2000/2000
TFRecord file created！
